In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel, GenSeq}
import org.clustering4ever.scala.clustering.kcenters.{KMeans, KMeansArgs}
import org.clustering4ever.spark.clustering.kcenters.{KMeans => KMeansSpark, KMeansArgs => KMeansArgsSpark}
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.indices.{ExternalIndicesLocal, InternalIndicesLocal}
import org.clustering4ever.enums.NmiNormalizationNature._
import org.clustering4ever.enums.ExternalsIndices
import org.clustering4ever.enums.InternalsIndices
import org.clustering4ever.util.ScalaCollectionImplicits._
import org.clustering4ever.clusterizables.EasyClusterizable
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4ever.vectors.ScalarVector
import org.clustering4ever.vectorizations.{EasyVectorization, VectorizationNature, Scalar, Default}
import org.clustering4ever.scala.clustering.rla.{RLAArgs, RLAArgsScalar, RLA, RLAScalar}
import org.clustering4ever.shapeless.{VMapping, VectorizationMapping}
import org.clustering4ever.clustering.chaining.{ClusteringChainingLocal, ClusteringChainingDistributed, ClusteringChainingClusteredSystem}
import scala.collection.parallel
import scala.collection.parallel.mutable.ParArray
import org.clustering4ever.clustering.indices.ClustersIndicesAnalysisLocal

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel, GenSeq}
import org.clustering4ever.scala.clustering.kcenters.{KMeans, KMeansArgs}
import org.clustering4ever.spark.clustering.kcenters.{KMeans=>KMeansSpark, KMeansArgs=>KMeansArgsSpark}
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.indices.{ExternalIndicesLocal, InternalIndicesLocal}
import org.clustering4ever.enums.NmiNormalizationNature._
import org.clustering4ever.enums.ExternalsIndices
import org.clustering4ever.enums.InternalsIndices
import org.clustering4ever.util.ScalaCollectionImplicits._
import org.clustering4ever.clusterizables.EasyClusterizable
import org.clustering4ever.vectorizables.Vectorizable
import org.clustering4...

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2019-01-15 22:57:04--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.6’

     0K .......                                               100% 6.08M=0.001s

2019-01-15 22:57:04 (6.08 MB/s) - ‘/tmp/aggregation.csv.6’ saved [8063/8063]

:sh: Scheme missing.
--2019-01-15 22:57:04--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-01-15 22:57:04--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.6’

     0K .                                                    

In [ ]:
val datasetSize = 100000
val dim = 10
val path = "/tmp/aggregation.csv"
val useAggregationDS = true

val rawData = if(useAggregationDS) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))
val groundTruePath = "/tmp/labels"

datasetSize: Int = 100000
dim: Int = 10
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = true
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Double]] = ArrayBuffer(ArrayBuffer(15.55, 28.65), ArrayBuffer(14.9, 27.55), ArrayBuffer(14.45, 28.35), ArrayBuffer(14.15, 28.8), ArrayBuffer(13.75, 28.05), ArrayBuffer(13.35, 28.45), ArrayBuffer(13.0, 29.15), ArrayBuffer(13.45, 27.5), ArrayBuffer(13.6, 26.5), ArrayBuffer(12.8, 27.35), ArrayBuffer(12.4, 27.85), ArrayBuffer(12.3, 28.4), ArrayBuffer(12.2, 28.65), ArrayBuffer(13.4, 25.1), ArrayBuffer(12.95, 25.95), ArrayBuffer(12.9, 26.5), ArrayBuffer(11.85, 27.0), ArrayBuffer(11.35, 28.0), ArrayBuffer(11.15, 28.7), ArrayBuffer(11.25, 27.4), ArrayBuffer(10.75, 27.7), ArrayBuffer(10.5, 28.35), ArrayBuffer(9....

## Transform your dataset adequately 

In [ ]:
val data = rawData.zipWithIndex.toVector.map{ case (v, id) => EasyClusterizable(id.toLong, ScalarVector(v)) }
val rddData = sc.parallelize(data.seq, 10)

data: scala.collection.immutable.Vector[org.clustering4ever.clusterizables.EasyClusterizable[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]]] = Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@91d1eee,Vector()), EasyClusterizable(1,Vectorizable(ScalarVector(ArrayBuffer(14.9, 27.55))),ScalarVector(ArrayBuffer(14.9, 27.55)),shapeless.HMap@353108a8,Vector()), EasyClusterizable(2,Vectorizable(ScalarVector(ArrayBuffer(14.45, 28.35))),ScalarVector(ArrayBuffer(14.45, 28.35)),shapeless.HMap@57ebb1e3,Vector()), EasyClusterizable(3,Vectorizable(ScalarVector(ArrayBuffer(14.15, 28.8)...

## Let's try two metrics

### For every metrics you have to precise the nature of your vector (List[Int], Vector[Double], mutable.ArrayBuffer[Int],...)

In [ ]:
val metric = Euclidean[mutable.ArrayBuffer[Double]](false)
val metric2 = Minkowski[mutable.ArrayBuffer[Double]](4)

metric: org.clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance without squared root applied
metric2: org.clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = Minkowski(4,3)


## Instantiae the clustering chaining tool

In [ ]:
val cc = new ClusteringChainingLocal(data)
val ccrdd = new ClusteringChainingDistributed(rddData)
val ccsmallOnDistributedSystem = new ClusteringChainingClusteredSystem(sc, data)

val firstVecto = cc.currentVectorization

cc: org.clustering4ever.clustering.chaining.ClusteringChainingLocal[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.clusterizables.EasyClusterizable,scala.collection.immutable.Vector] = ClusteringChainingLocal(Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@91d1eee,Vector()), EasyClusterizable(1,Vectorizable(ScalarVector(ArrayBuffer(14.9, 27.55))),ScalarVector(ArrayBuffer(14.9, 27.55)),shapeless.HMap@353108a8,Vector()), EasyClusterizable(2,Vectorizable(ScalarVector(ArrayBuffer(14.45, 28.35))),ScalarVector(ArrayBuffer(14.45, 28.35)),shapeless.HMap@57ebb...

## Two ways to prepare algorithm to test
### The painfull one

In [ ]:
val rlaArgs = RLAArgsScalar(metric, 2D)
val kMeansArgs = KMeansArgs(2, metric, 0.0001, 50)
val kMeansArgsSpark = KMeansArgsSpark(2, metric, 0.0001, 50)
val kMeans = KMeans[
    Long,
    ScalarVector[mutable.ArrayBuffer[Double]],
    mutable.ArrayBuffer[Double],
    EasyClusterizable,
    Euclidean,
    immutable.Vector
](kMeansArgs)
val rla = RLAScalar[
    Long,
    ScalarVector[mutable.ArrayBuffer[Double]],
    mutable.ArrayBuffer[Double],
    EasyClusterizable,
    Euclidean,
    immutable.Vector
](rlaArgs)

val kMeansVariousK = (3 to 9).map{ i =>
  KMeans[
    Long,
    ScalarVector[mutable.ArrayBuffer[Double]],
    mutable.ArrayBuffer[Double],
    EasyClusterizable,
    Euclidean,
    immutable.Vector
  ](
    KMeansArgs(i, metric, 0.0001, 50)
  )
}

val kMeansVariousKSpark = (3 to 9).map{ i =>
  KMeansSpark[
    Long,
    ScalarVector[mutable.ArrayBuffer[Double]],
    mutable.ArrayBuffer[Double],
    EasyClusterizable,
    Euclidean
  ](
    KMeansArgsSpark(i, metric, 0.0001, 50)
  )
}

val algosObtainedPainfully = Seq(kMeans, rla) ++ kMeansVariousK
val algosObtainedSemiEffortlesslySpark = kMeansVariousKSpark :+ kMeansArgsSpark.obtainAlgorithm(rddData)

rlaArgs: org.clustering4ever.scala.clustering.rla.RLAArgsScalar[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = RLAArgsScalar(Euclidean distance without squared root applied,2.0)
kMeansArgs: org.clustering4ever.scala.clustering.kcenters.KMeansArgs[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = KMeansArgs(2,Euclidean distance without squared root applied,1.0E-4,50,Map())
kMeansArgsSpark: org.clustering4ever.spark.clustering.kcenters.KMeansArgs[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.scalar.Euclidean] = KMeansArgs(2,Euclidean distance without squared root applied,1.0E-4,50,StorageLevel(memory, deserialized, 1 replicas),Map())
kMeans: org.clusteri...

## Easily using right methods

In [ ]:
val kmeansArgsCombinations = KMeans.generateAnyArgumentsCombination(
  (4 until 100), Seq(metric, metric2), Seq(5D), Seq(50, 100, 200), Seq(mutable.HashMap.empty[Int, ScalarVector[mutable.ArrayBuffer[Double]]])
)

val othersKmeansArgsCombinations = (23 to 30).map( i => KMeansArgs(i, metric, 0.0001, 50) )

val kmeansAlgos = kmeansArgsCombinations.map(_.obtainAlgorithm(data)).toBuffer
val others = othersKmeansArgsCombinations.map(_.obtainAlgorithm(data)).toSeq

kmeansArgsCombinations: Seq[org.clustering4ever.scala.clustering.kcenters.KMeansArgs[scala.collection.mutable.ArrayBuffer[Double],org.clustering4ever.math.distances.ContinuousDistance]] = Vector(KMeansArgs(4,Euclidean distance without squared root applied,5.0,50,Map()), KMeansArgs(4,Euclidean distance without squared root applied,5.0,100,Map()), KMeansArgs(4,Euclidean distance without squared root applied,5.0,200,Map()), KMeansArgs(4,Minkowski(4,3),5.0,50,Map()), KMeansArgs(4,Minkowski(4,3),5.0,100,Map()), KMeansArgs(4,Minkowski(4,3),5.0,200,Map()), KMeansArgs(5,Euclidean distance without squared root applied,5.0,50,Map()), KMeansArgs(5,Euclidean distance without squared root applied,5.0,100,Map()), KMeansArgs(5,Euclidean distance without squared root applied,5.0,200,Map()), KMeansArgs(...

## Local Algorithms launched multiple algorithms PARALLELY on local machine

In [ ]:
val t0 = System.currentTimeMillis
val cc2 = cc.runAlgorithms(kmeansAlgos:_*)
val t1 = System.currentTimeMillis
val cc3 = cc2.runAlgorithms(others:_*)
val t2 = System.currentTimeMillis
(t2 - t0) / 1000000D

t0: Long = 1547589554759
cc2: org.clustering4ever.clustering.chaining.ClusteringChainingLocal[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.clusterizables.EasyClusterizable,scala.collection.immutable.Vector] = ClusteringChainingLocal(Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@91d1eee,Vector(3, 1, 0, 2, 2, 1, 0, 2, 4, 2, 0, 4, 3, 4, 2, 4, 4, 2, 2, 3, 5, 1, 0, 0, 4, 2, 7, 3, 1, 0, 1, 1, 2, 7, 1, 4, 4, 5, 4, 5, 6, 6, 3, 7, 3, 7, 2, 8, 6, 0, 11, 5, 11, 3, 12, 4, 2, 3, 8, 12, 9, 8, 10, 8, 0, 1, 12, 6, 8, 1, 10, 5, 1, 6, 0, 7, 14, 4, 15, 1, 8, 15, 7, ...

0.018653

In [ ]:
kmeansAlgos.size
cc3.currentVectorization.clusteringNumbers.size

res44: Int = 584


584

## Distributed Algorithms launch sequentially on a distributed system 

In [ ]:
val t0 = System.currentTimeMillis
val ccrdd2 = ccrdd.runAlgorithms(algosObtainedSemiEffortlesslySpark:_*)
val t1 = System.currentTimeMillis
(t1 - t0) / 1000000D

t0: Long = 1547589609566
ccrdd2: org.clustering4ever.clustering.chaining.ClusteringChainingDistributed[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.clusterizables.EasyClusterizable] = org.clustering4ever.clustering.chaining.ClusteringChainingDistributed$$anon$2@6acc6421
t1: Long = 1547589625351
res49: Double = 0.015785


0.015785

In [ ]:
ccrdd2.data.first.clusterIDs

res51: scala.collection.immutable.Vector[Int] = Vector(1, 0, 4, 3, 5, 7, 1, 0)


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

## Local Algorithms launched parallely on a distributed system

In [ ]:
val t0 = System.currentTimeMillis
val ccsmallOnDistributedSystem2 = ccsmallOnDistributedSystem.runAlgorithms(10, kmeansAlgos:_*)
val t1 = System.currentTimeMillis
(t1 - t0) / 1000000D

t0: Long = 1547589677807
ccsmallOnDistributedSystem2: ccsmallOnDistributedSystem.Self[org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]] = org.clustering4ever.clustering.chaining.ClusteringChainingClusteredSystem$$anon$8@79501fe9
t1: Long = 1547589707905
res56: Double = 0.030098


0.030098

In [ ]:
ccsmallOnDistributedSystem2.data.head.clusterIDs.size

576

res58: Int = 576


576

## Each run release a new clustering chaining object, with updated values

In [ ]:
val t0 = System.currentTimeMillis
val newCC = cc3.runAlgorithms(algosObtainedPainfully:_*)
val t1 = System.currentTimeMillis
(t1 - t0) / 10000000D

t0: Long = 1547589709517
newCC: org.clustering4ever.clustering.chaining.ClusteringChainingLocal[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.clusterizables.EasyClusterizable,scala.collection.immutable.Vector] = ClusteringChainingLocal(Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@91d1eee,Vector(3, 1, 0, 2, 2, 1, 0, 2, 4, 2, 0, 4, 3, 4, 2, 4, 4, 2, 2, 3, 5, 1, 0, 0, 4, 2, 7, 3, 1, 0, 1, 1, 2, 7, 1, 4, 4, 5, 4, 5, 6, 6, 3, 7, 3, 7, 2, 8, 6, 0, 11, 5, 11, 3, 12, 4, 2, 3, 8, 12, 9, 8, 10, 8, 0, 1, 12, 6, 8, 1, 10, 5, 1, 6, 0, 7, 14, 4, 15, 1, 8, 15, 7...

3.8E-6

## Get some informations about your process

In [ ]:
newCC.data.last.clusterIDs
newCC.globalClusteringRunNumber
newCC.clusteringInfo.clusteringInformations.take(2)
// newCC.currentVectorization.clusteringNumbers.toList.sorted.size

<console>:152: warning: a pure expression does nothing in statement position; you may be omitting necessary parentheses
       newCC.globalClusteringRunNumber
             ^
res62: scala.collection.immutable.Vector[(org.clustering4ever.types.ClusteringInformationTypes.GlobalClusteringRunNumber, org.clustering4ever.vectorizations.Vectorization[org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]], _], org.clustering4ever.clustering.ClusteringArgsLocal[_], org.clustering4ever.clustering.ClusteringModelLocal[Long, org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]], _, org.clustering4ever.clusterizables.EasyClusterizable, scala.collection.immutable.Vector, org.clustering4ever.clustering.ClusteringArgsLocal[_]])] = Vector((0,EasyVectorization(0,None,Set(),Vector()),KMeansArgs(4,Euclidean distance without squared root applied,5.0,50,Map()),KMeansModel(Map(2 -> ScalarVector(ArrayBuffer(23.3718905...

entries total 
 
 
 
 
 
 
 
 
 entries total

## Lets explore the clustering indices

In [ ]:
val indicesAnalysis = new ClustersIndicesAnalysisLocal(newCC.data)

indicesAnalysis: org.clustering4ever.clustering.indices.ClustersIndicesAnalysisLocal[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.clusterizables.EasyClusterizable,scala.collection.immutable.Vector] = ClustersIndicesAnalysisLocal(Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@91d1eee,Vector(3, 1, 0, 2, 2, 1, 0, 2, 4, 2, 0, 4, 3, 4, 2, 4, 4, 2, 2, 3, 5, 1, 0, 0, 4, 2, 7, 3, 1, 0, 1, 1, 2, 7, 1, 4, 4, 5, 4, 5, 6, 6, 3, 7, 3, 7, 2, 8, 6, 0, 11, 5, 11, 3, 12, 4, 2, 3, 8, 12, 9, 8, 10, 8, 0, 1, 12, 6, 8, 1, 10, 5, 1, 6, 0, 7, 14, 4, 15, 1, 8, 15, 7, 13, ...

In [ ]:
val internalIndices = InternalsIndices.values.toSeq.init
val extScores = indicesAnalysis.saveInternalsIndicesForEveryClusteringNumber(metric, internalIndices:_*)

val clusteringNumber = 5
// It is a tuple 3 as key, (MetricID, clusteringNumber, IndexEnum)
extScores.internalsIndicesByMetricClusteringNumberIndex.get(metric.id, clusteringNumber, internalIndices.head)

val oneInternalIndex = internalIndices.head

val first100ResultForMetricAndFirstInternalIndex =
  (0 until 100).map( i => extScores.internalsIndicesByMetricClusteringNumberIndex.get(metric.id, i, oneInternalIndex) )

first100ResultForMetricAndFirstInternalIndex
// extScores.internalsIndicesByMetricClusteringNumberIndex.get(metric.id, clusteringNumber, internalIndices.head)


internalIndices: Seq[org.clustering4ever.enums.InternalsIndices.Value] = ArrayBuffer(DaviesBouldin, BallHall)
extScores: org.clustering4ever.clustering.indices.ClustersIndicesAnalysisLocal[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.clusterizables.EasyClusterizable,scala.collection.immutable.Vector] = ClustersIndicesAnalysisLocal(Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(15.55, 28.65)),shapeless.HMap@91d1eee,Vector(3, 1, 0, 2, 2, 1, 0, 2, 4, 2, 0, 4, 3, 4, 2, 4, 4, 2, 2, 3, 5, 1, 0, 0, 4, 2, 7, 3, 1, 0, 1, 1, 2, 7, 1, 4, 4, 5, 4, 5, 6, 6, 3, 7, 3, 7, 2, 8, 6, 0, 11, 5, 11...

entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
val gt = Source.fromFile(groundTruePath).getLines.map(_.toInt).toVector
val everyExternalIndexes = ExternalsIndices.values.toSeq
indicesAnalysis.computeExternalsIndicesForEveryClusteringNumber(gt, everyExternalIndexes:_*)

gt: Vector[Int] = Vector(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4...

entries total

## Define a new vectorization 

In [ ]:
def newVecto(v: ScalarVector[mutable.ArrayBuffer[Double]]) = new ScalarVector(v.vector.toVector)

def newVecto2(v: ScalarVector[mutable.ArrayBuffer[Double]]) = new ScalarVector(v.vector.tail)

val vectorizationNature = Scalar

val vectorization1 = EasyVectorization(1, Some(newVecto _))
val vectorization2 = EasyVectorization(2, Some(newVecto2 _))

// Add vectorization of another kind that current one
val (cc3, implicitMappingTowardVector) = newCC.addAnyVectorization(vectorization1)
// Add vectorization of same type that current one
val (cc4, implicitMappingTowardVector2) = cc3.addAnyVectorization(vectorization2)
// Update current vector for same type that current one
val cc5 = cc4.updateAnyVector(vectorization2)

(cc4.data.head.v, cc5.data.head.v)

newVecto: (v: org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]])org.clustering4ever.vectors.ScalarVector[Vector[Double]]
newVecto2: (v: org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]])org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]]
vectorizationNature: org.clustering4ever.vectorizations.Scalar.type = org.clustering4ever.vectorizations.Scalar$@11cc1bd3
vectorization1: org.clustering4ever.vectorizations.EasyVectorization[org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[Vector[Double]]] = EasyVectorization(1,Some(<function1>),Set(),Vector())
vectorization2: org.clustering4ever.vectorizations.EasyVec...

(ScalarVector(ArrayBuffer(15.55, 28.65)),ScalarVector(ArrayBuffer(28.65)))

In [ ]:
// Update current vector for different type that current one
val cc6 = cc5.updateAnyVector(vectorization1)
cc6.data.head.v

cc6: org.clustering4ever.clustering.chaining.ClusteringChainingLocal[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[Vector[Double]],org.clustering4ever.clusterizables.EasyClusterizable,scala.collection.immutable.Vector] = ClusteringChainingLocal(Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(Vector(15.55, 28.65)),shapeless.HMap@79b31012,Vector(3, 1, 0, 2, 2, 1, 0, 2, 4, 2, 0, 4, 3, 4, 2, 4, 4, 2, 2, 3, 5, 1, 0, 0, 4, 2, 7, 3, 1, 0, 1, 1, 2, 7, 1, 4, 4, 5, 4, 5, 6, 6, 3, 7, 3, 7, 2, 8, 6, 0, 11, 5, 11, 3, 12, 4, 2, 3, 8, 12, 9, 8, 10, 8, 0, 1, 12, 6, 8, 1, 10, 5, 1, 6, 0, 7, 14, 4, 15, 1, 8, 15, 7, 13, 7, 7, 2, 9, 6, 8, 10, 13, 6, 17, 3, 7, 6, 10, 6, 14, 7,...

ScalarVector(Vector(15.55, 28.65))

In [ ]:
val metricVector = new Euclidean[Vector[Double]](false)


val generateAlgoForList = (23 to 30).map( i => KMeansArgs(i, metricVector, 0.0001, 50) ).map(_.obtainAlgorithm(cc6.data))

val cc7 = cc6.runAlgorithms(generateAlgoForList:_*)

metricVector: org.clustering4ever.math.distances.scalar.Euclidean[Vector[Double]] = Euclidean distance without squared root applied
generateAlgoForList: scala.collection.immutable.IndexedSeq[org.clustering4ever.scala.clustering.kcenters.KMeans[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],Vector[Double],org.clustering4ever.clusterizables.EasyClusterizable,org.clustering4ever.math.distances.scalar.Euclidean,scala.collection.immutable.Vector]] = Vector(KMeans(KMeansArgs(23,Euclidean distance without squared root applied,1.0E-4,50,Map())), KMeans(KMeansArgs(24,Euclidean distance without squared root applied,1.0E-4,50,Map())), KMeans(KMeansArgs(25,Euclidean distance without squared root applied,1.0E-4,50,Map())), KMeans(KMeansArgs(26,Euclidean d...

In [ ]:
val cc8 = cc7.updateAnyVector(vectorization2)

cc8.data.head.clusterIDs

cc8: org.clustering4ever.clustering.chaining.ClusteringChainingLocal[Long,org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.vectors.ScalarVector[scala.collection.mutable.ArrayBuffer[Double]],org.clustering4ever.clusterizables.EasyClusterizable,scala.collection.immutable.Vector] = ClusteringChainingLocal(Vector(EasyClusterizable(0,Vectorizable(ScalarVector(ArrayBuffer(15.55, 28.65))),ScalarVector(ArrayBuffer(28.65)),shapeless.HMap@79b31012,Vector(3, 1, 0, 2, 2, 1, 0, 2, 4, 2, 0, 4, 3, 4, 2, 4, 4, 2, 2, 3, 5, 1, 0, 0, 4, 2, 7, 3, 1, 0, 1, 1, 2, 7, 1, 4, 4, 5, 4, 5, 6, 6, 3, 7, 3, 7, 2, 8, 6, 0, 11, 5, 11, 3, 12, 4, 2, 3, 8, 12, 9, 8, 10, 8, 0, 1, 12, 6, 8, 1, 10, 5, 1, 6, 0, 7, 14, 4, 15, 1, 8, 15, 7, 13, 7, 7, 2, 9, 6, 8, 10, 13, 6...

entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
cc8.vectorizations.get(vectorization1.vectorizationID)(vectorization1.vectoMapping).get.clusteringNumbers.toSeq.sorted

res95: Seq[Int] = ArrayBuffer(593, 594, 595, 596, 597, 598, 599, 600)


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
cc8.vectorizations.get(firstVecto.vectorizationID)(firstVecto.vectoMapping).get.clusteringNumbers.toSeq.sorted

res97: Seq[Int] = ArrayBuffer(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 17...

entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total